# MongoDB的shell命令

## mongodbshell命令

In [ ]:
# 查看版本号
mongo --version

use testdb

db.auth('arvin', 'arvin@123456.')

### python执行mongo的shell命令

In [19]:
print(db.command("dbstats")) 
print(db.name)
# db.command("collstats", "test_collection") # prints collection-level stats for "test_collection" under "test_db". 

{'db': 'testdb', 'collections': 0, 'views': 0, 'objects': 0, 'avgObjSize': 0, 'dataSize': 0, 'storageSize': 0, 'numExtents': 0, 'indexes': 0, 'indexSize': 0, 'fileSize': 0, 'fsUsedSize': 0, 'fsTotalSize': 0, 'ok': 1.0}
testdb


### 数据导入与导出

In [ ]:
import subprocess
from pymongo import MongoClient

def export_data(db, dst_path, param):
    for collection in db.collection_names():
        log.info("开始导出 {} 数据库 {} ...".format(db, collection))
        try:
            subprocess.check_call(["mongoexport",
                                   "-h", param["host"],
                                   "-u", param["user"],
                                   "-p", param["passwd"],
                                   "-d", db,
                                   "-c", collection,
                                   "-o", dst_path])
            log.info("导出 {} 数据成功".format(dst_path))
        except Exception as e:
            log.error("导出MongoDB数据出错,数据导出中止, Error: {}".format(e))

In [ ]:
import subprocess
import pymongo
from pymongo import MongoClient

# 导入与导出数据
# mongoimport -d 数据库名字 -c 集合名字 --type 文件类型 --file 文件路径  --upsert,
# 其中，--upsert字段的意思是以插入(insert)或者更新(update)的方式来导入数据。
# 注意：如果数据库存在数据，要导入最新的数据，则需要加--upsert选项，会更新数据，否则会报错(提示重复键错误收集)
def importdata(db_name, collection, file_name, datatype='json'):
    try:
        subprocess.Popen(['mongoimport', '-d', db_name, '-c', collection,
                         '--type', datatype, '--file', file_name, '--upsert'])
    except Exception as e:
        print("导入数据异常:{}".format(e))

# 导入数据到远程服务器上
# mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --upsert --drop 文件名的具体路径   (--drop当不需要删除源文件可不加)
def importdata(user, passwd, db_name, collection, file_name, host='127.0.0.1', port='27017', datatype='json'):
    try:
        subprocess.Popen(['mongoimport', '-h', host, '--port', port, 
                          '-u', user, '-p', passwd, '-d', db_name, '-c', collection,
                          '--type', datatype, '--file', file_name, '--upsert'])
    except Exception as e:
        print("导入数据异常:{}".format(e))

# 导入部分字段到collection中
# 命令格式：mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --upsertFields 字段 --drop 文件名的具体路径 (--drop当不需要删除源文件可不加)

# 导入csv文件
# 命令格式：mongoimport -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -c 表名 --type 类型(csv) --headerline --upsert --drop 文件名的具体路径  

def exportdata(host, user, passwd, db_name, collection, dst):
    try:
        subprocess.Popen(['mongoexport', '-h', host, '-u', user,
                         '-p', passwd, '-d', db_name, '-c', collection, '-o', dst])
    except Exception as e:
        print("导出数据异常:{}".format(e))

# 数据备份与还原
# 备份命令: mongodump -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 -o 文件存放路径
# 还原命令: mongorestore -h IP --port 端口 -u 用户名 -p 密码 -d 数据库 --drop 文件存在路径,--drop是先删除所有的数据，再恢复，不需要删除可不加;

def dumpdata(host, user, passwd, db_name, collection, dst):
    try:
        subprocess.Popen(['mongodump', '-h', host, '-u', user,
                         '-p', passwd, '-d', db_name, '-c', collection, '-o', dst])
    except Exception as e:
        print("备份数据异常:{}".format(e))

def restoredata(host, user, passwd, db_name, collection, restorepath)
    try:
        subprocess.Popen(['mongoretore', '-h', host, '-u', user,
                          '-p', passwd, '-d', db_name, '-c', collection, restorepath])
    except Exception as e:
        print("还原数据异常:{}".format(e))

# 建立连接, 返回数据库对象
def connect_db(user, passwd, host, db_name):
    client = MongoClient(
        'mongodb://{}:{}@{}/{}'.format(user, passwd, host, db_name))
    return client[db_name]


# 获取数据库中所有集合名
db = connect_db(user, passwd, host, db_name)
collections = db.collection_names()

collection_name = "xxxx"
movie = db[collection_name]
# 查询

# 范围查询
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    print(post)

# 正则匹配查询
# 查询某一字段不是数字的记录
db.getCollection('phone').find({"tel": {"$regex": '^[^0-9]+$'}})
# 查询某一字段不是数字的记录的数目
db.getCollection('phone').find({"tel": {"$regex": '^[^0-9]+$'}}).count()
# 删除某一字段不是数字的记录
db.getCollection('phone').remove({"tel": {"$regex": '^[^0-9]+$'}})

# 查询某个字段多个值的数据
db.collection.find({"A": {"$in": [1, 2, 3]}})

# 查询两个字段值相同的记录
db.foo.find({"$where":function(){
 for(var current in this){
   for(var other in this){
     if(current != other && this[current] == this[other]){
       return true;
     }
   }
 }
 return false;
}})

# 查询给定字段有重复数据的记录
db.collection.aggregate([
  { $group: { 
    _id: { page: "$page", secondField: "$secondField" }, 
    uniqueIds: { $addToSet: "$_id" },
    count: { $sum: 1 } 
  }}, 
  { $match: { 
    count: { $gt: 1 } 
  }}
])

# 简单去重方法,适合数据量不大的情况下, 返回去重后的列表
db.collection.distinct("page")

# 求某字段最大值：#得到的记录为集合中"_id"值最大的那一条
collection.find().sort({"_id":-1}).limit(1)
# 统计不含有某一字段的记录数：
dbName.collectionName.find({fieldName:null}).count()
# 查询结果排序 , --默认为升序
movie.find().sort("UserName")
movie.find().sort("UserName", pymongo.ASCENDING)
movie.find().sort("UserName", pymongo.DESCENDING)
# 查询结果多列排序
movie.find().sort(["UserName", pymongo.DESCENDING), ("Email", pymongo.DESCENDING)])

# 修改/更新记录
movie.update({"UserName": "xxx"}, {
             "$set": {"Email": "xxxx", "Password": "xxx"}})
# 增加字段：
collection.update({"_id":1},{"$set":{"new_field":0}})

# 删除
# -- 全部删除, 慎用
movie.remove({})
movie.remove({"UserName": "xxxxx"})
# 删除字段：
collection.update({"_id":1},{"$unset":{"new_field":1}})

# 索引（Indexing）
# 为了让上述查询更快一点，可以添加一个在"date” 和 “author"上添加复合索引。
# 首先，使用explain()方法来了解查询在没有添加索引情况下如何执行:
movie.find({"date": {"$lt": d}}).sort("author").explain()["cursor"]
>> > u'BasicCursor'

movie.find({"date": {"$lt": d}}).sort("author").explain()["nscanned"]
>> > 3
# 查询使用的是BasicCursor,而且扫描了全部的三个文件。现在添加一个复合索引，再看看同样的操作：
# 创建data和author索引
movie.create_index([("date", DESCENDING), ("author", ASCENDING)])
>> > u'date_-1_author_1'
movie.find({"date": {"$lt": d}}).sort("author").explain()["cursor"]
>> > u'BtreeCursor date_-1_author_1'
movie.find({"date": {"$lt": d}}).sort("author").explain()["nscanned"]
>> > 2
# 现在查询使用的是BtreeCursor(利用这个索引)，并且只扫描了两个符合条件的文件。

# map_reduce
# map函数的作用是遍历集合，调用emit函数将集合中userId、nick字段以键值对的形式传递给reduce函数
mapper = Code("""
function(){
    emit(this.userId, this.nick);
}
""")

# reduce函数的作用对map函数传递过来的键值对进行处理, 
# 每个<key, values>键值对中，key是userId字段的值，values是具有相同userId的nick的数组。
# 由于我的程序中一个values的各个元素的值是相同的，所以没有对values进行遍历。
reducer = Code("""
function(key, values){
    var result = {nick: values[0]};
    return result;
}
""")

# 启动MapReduce, 将结果输出到seller_info这个集合中
result = collection.map_reduce(mapper, reducer, 'seller_info')

# 遍历seller_info集合，查看MapReduce结果
for doc in db['seller_info'].find():
    print doc
